In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import sklearn

In [2]:
import scanpy as sc
import squidpy as sq

In [1]:
import sklearn.ensemble
import sklearn.linear_model

In [4]:
adata = sc.read_h5ad("../dplfc_data/adata_visium.h5ad")

In [5]:
import torch

adata_list = []
for idx in list(adata.obs['sample_id'].unique()):
    adata_s = adata[adata.obs['sample_id'] == str(idx)]
    emb_data = torch.load(f"../UNI/visium_{idx}_allspot_uni_200.pkl")
    emb_data = np.array(emb_data)
    adata_s.obsm['X_emb'] = emb_data
    adata_list.append(adata_s)

adata_gpfm = sc.concat(adata_list)

In [ ]:
result_list = []
for i in list(adata.obs['sample_id'].unique()):
    adata_train = adata_gpfm[adata_gpfm.obs['sample_id'] != i]
    adata_test = adata_gpfm[adata_gpfm.obs['sample_id'] == i]
    adata_train = adata_train[adata_train.obs['cluster'] != 'nan']
    adata_test = adata_test[adata_test.obs['cluster'] != 'nan']
    
    
    adata_train.X = adata_train.layers['logcounts']
    sc.pp.highly_variable_genes(adata_train,batch_key='sample_id')
    adata_train = adata_train[:,adata_train.var['highly_variable']]
    
    adata_test.X = adata_test.layers['logcounts']
    adata_test = adata_test[:,adata_train.var_names]

    adata_train.obsm['X_emb_comb'] = np.concatenate((adata_train.X.toarray(), adata_train.obsm['X_emb']), axis=1)
    adata_test.obsm['X_emb_comb'] =  np.concatenate((adata_test.X.toarray(), adata_test.obsm['X_emb']), axis=1)
    
    model = sklearn.linear_model.LogisticRegression(n_jobs=-1)
    model.fit(adata_train.obsm['X_emb_comb'], adata_train.obs['cluster'].values)
    pred = model.predict(adata_test.obsm['X_emb_comb'])
    true_label = adata_test.obs['cluster'].values
    dict_out = sklearn.metrics.classification_report(true_label,pred,digits=4, output_dict=True)
    single_out = [dict_out['accuracy'], dict_out['weighted avg']['precision'] , dict_out['weighted avg']['recall'] , dict_out['weighted avg']['f1-score']]
    result_list.append(single_out)

In [ ]:
for i in result_list:
    print(i[0], ' ', i[1], ' ', i[2], ' ', i[3])